# **Chapter 9: Non-Comparison Sorting**

> *"If you know something about your data, you can sort faster than O(n log n)."* — Robert Sedgewick

---

## **9.1 Introduction**

While comparison-based sorts are limited to $\Omega(n \log n)$ time complexity (as proven by the decision tree model in Chapter 8), **non-comparison sorts** break this barrier by exploiting specific properties of the input data—such as integer ranges, digit structure, or distribution characteristics.

These algorithms achieve **linear time complexity** $O(n)$ under specific conditions, making them indispensable for large-scale data processing where integers or fixed-length keys are involved.

```
┌─────────────────────────────────────────────────────────────────────┐
│         COMPARISON vs NON-COMPARISON SORTING                         │
├─────────────────────────────────────────────────────────────────────┤
│                                                                      │
│  Comparison Sorts:                    Non-Comparison Sorts:         │
│  ─────────────────                    ─────────────────────         │
│  • Work on any comparable type        • Work on specific types       │
│    (integers, strings, objects)         (integers, fixed-length     │
│                                          keys, uniformly distributed │
│  • Lower bound: Ω(n log n)              data)                        │
│  • General purpose                    • Linear time O(n) possible   │
│                                       • Requires extra assumptions   │
│                                       • Not general purpose          │
│                                                                      │
│  When to use Non-Comparison:                                         │
│  • Sorting integers in known range [0, k] → Counting Sort           │
│  • Sorting fixed-length strings/numbers → Radix Sort                │
│  • Data uniformly distributed → Bucket Sort                          │
│  • Data too large for memory → External Sort                        │
│                                                                      │
└─────────────────────────────────────────────────────────────────────┘
```

---

## **9.2 Counting Sort**

Counting Sort assumes each input element is an integer in the range $[0, k]$ where $k = O(n)$. It determines, for each input element $x$, the number of elements less than $x$ and uses this information to place $x$ directly into its correct position.

### **9.2.1 Algorithm Theory**

```python
from typing import List, Tuple
import random

def counting_sort(arr: List[int], 
                  max_val: int = None,
                  stable: bool = True) -> List[int]:
    """
    Counting Sort - Linear time integer sorting.
    
    Algorithm:
      1. Count occurrences of each value (frequency array)
      2. Compute prefix sums (cumulative counts)
      3. Place elements in output array using prefix sums
    
    Time Complexity:
      Best/Average/Worst: O(n + k) where k is range of input
      If k = O(n), then O(n) linear time
    
    Space Complexity: O(n + k)
      • O(k) for count array
      • O(n) for output array (if stable)
    
    Stability: Yes (if implemented with prefix sum scan right-to-left)
    
    Constraints:
      • Input must be integers (or convertible to integers)
      • Must know range [0, k] beforehand (or find min/max first)
      • Inefficient if k >> n (e.g., sorting two elements with values 0 and 1,000,000)
    
    Args:
        arr: List of non-negative integers to sort
        max_val: Maximum value in array (if None, computed)
        stable: If True, maintains relative order (requires extra space)
    
    Returns:
        New sorted list (does not sort in-place)
    """
    if not arr:
        return []
    
    n = len(arr)
    
    # Find range if not provided
    if max_val is None:
        max_val = max(arr)
        min_val = min(arr)
    else:
        min_val = 0  # Assume non-negative if max provided
    
    # Handle negative numbers by shifting
    if min_val < 0:
        # Shift all values to be non-negative
        offset = -min_val
        arr = [x + offset for x in arr]
        max_val = max_val + offset
    else:
        offset = 0
    
    k = max_val + 1  # Range is [0, max_val]
    
    # Step 1: Initialize count array
    # count[i] will hold the number of occurrences of value i
    count = [0] * k
    
    # Step 2: Count occurrences
    for num in arr:
        count[num] += 1
    
    if not stable:
        # Non-stable version (overwrites input, uses less space)
        # Simply expand counts into output
        idx = 0
        for value in range(k):
            for _ in range(count[value]):
                arr[idx] = value - offset  # Restore original value
                idx += 1
        return arr
    
    # Step 3: Compute prefix sums (cumulative counts)
    # count[i] now contains the number of elements <= i
    for i in range(1, k):
        count[i] += count[i - 1]
    
    # Step 4: Build output array (backwards for stability)
    # Going backwards ensures that elements with same value appear
    # in output in same order as input (stable sort)
    output = [0] * n
    
    for i in range(n - 1, -1, -1):
        value = arr[i]
        # Place element at position count[value] - 1
        output[count[value] - 1] = value - offset  # Restore original
        count[value] -= 1  # Decrement count for next occurrence
    
    return output


def counting_sort_optimized(arr: List[int]) -> List[int]:
    """
    Optimized counting sort that handles negative numbers
    and minimizes memory usage.
    """
    if not arr:
        return []
    
    # Find min and max to determine range
    min_val = min(arr)
    max_val = max(arr)
    
    # If range is too large relative to n, counting sort is inefficient
    range_size = max_val - min_val + 1
    n = len(arr)
    
    if range_size > n * 10 and range_size > 10000:
        print(f"Warning: Range {range_size} >> n {n}, consider other sort")
    
    # Offset to shift negative values to non-negative indices
    offset = -min_val
    
    # Count array
    count = [0] * range_size
    
    # Count occurrences
    for num in arr:
        count[num + offset] += 1
    
    # Build output
    output = []
    for value in range(range_size):
        original_value = value - offset
        output.extend([original_value] * count[value])
    
    return output


def demonstrate_counting_sort():
    """
    Demonstrate counting sort with various inputs.
    """
    print("Counting Sort Demonstration")
    print("=" * 70)
    
    # Test case 1: Small range
    arr1 = [4, 2, 2, 8, 3, 3, 1]
    print(f"Input:  {arr1}")
    print(f"Range: 0 to {max(arr1)}")
    sorted1 = counting_sort(arr1)
    print(f"Sorted: {sorted1}")
    
    # Test case 2: With negative numbers
    arr2 = [-5, -10, 0, -3, 8, 5, -1]
    print(f"\nWith negatives: {arr2}")
    sorted2 = counting_sort(arr2)
    print(f"Sorted:         {sorted2}")
    
    # Test case 3: Stability demonstration
    print("\nStability Test:")
    # Using tuples where first element is the sort key
    # In real use, we'd extract keys, sort indices, then rearrange
    print("Counting sort is stable - equal elements maintain relative order")
    
    # Test case 4: Large range warning
    arr3 = [1, 1000000]
    print(f"\nLarge range: {arr3}")
    print(f"Range size: {max(arr3) - min(arr3) + 1}, n: {len(arr3)}")
    print("Counting sort would allocate array of size 1,000,000 for 2 elements!")
    print("Use only when k = O(n)")
    
    print("""
    
    Counting Sort Analysis:
    ─────────────────────────────────────────────────────────────────────
    
    When to Use:
      • Integers in small, known range (e.g., ages 0-120, exam scores 0-100)
      • Sorting by digit in Radix Sort (k=10 for decimal)
      • Frequency counting problems
    
    When NOT to Use:
      • Large range with sparse data (memory waste)
      • Floating point numbers (without quantization)
      • Custom objects without integer keys
    
    Space Optimization:
      • If stability not required: O(k) space (modify input directly)
      • If stability required: O(n + k) space (output array needed)
    
    Time Breakdown:
      • Counting: O(n)
      • Prefix sums: O(k)
      • Placement: O(n)
      • Total: O(n + k)
    """)


demonstrate_counting_sort()
```

**Output:**
```
Counting Sort Demonstration
======================================================================
Input:  [4, 2, 2, 8, 3, 3, 1]
Range: 0 to 8
Sorted: [1, 2, 2, 3, 3, 4, 8]

With negatives: [-5, -10, 0, -3, 8, 5, -1]
Sorted:         [-10, -5, -3, -1, 0, 5, 8]

Stability Test:
Counting sort is stable - equal elements maintain relative order

Large range: [1, 1000000]
Range size: 1000000, n: 2
Counting sort would allocate array of size 1,000,000 for 2 elements!
Use only when k = O(n)
```

---

## **9.3 Radix Sort**

Radix Sort processes digits (or characters) from least significant to most significant (LSD) or vice versa (MSD). It uses Counting Sort as a subroutine to sort by each digit position.

### **9.3.1 LSD (Least Significant Digit) Radix Sort**

```python
def radix_sort_lsd(arr: List[int], 
                   base: int = 10) -> List[int]:
    """
    LSD (Least Significant Digit) Radix Sort.
    
    Algorithm:
      1. Find maximum number to determine digit count
      2. For each digit position (1s, 10s, 100s, ...):
         a. Sort numbers by current digit using counting sort
         b. Maintain stability (crucial for correctness)
    
    Time Complexity: O(d × (n + k)) where:
      • d = number of digits in maximum number
      • n = number of elements
      • k = base (e.g., 10 for decimal)
    
    If d is constant (fixed-length keys): O(n)
    If d = log_b(max): O(n log n) but with small constant
    
    Space: O(n + k) per digit pass
    
    Stability: Yes (must use stable sort for each digit)
    
    Constraints:
      • Integers only (or fixed-length strings)
      • Must handle negative numbers specially
    
    Args:
        arr: List of integers to sort
        base: Number base (10 for decimal, 2 for binary, 256 for bytes)
    
    Returns:
        Sorted list
    """
    if not arr or len(arr) <= 1:
        return arr.copy() if arr else []
    
    # Separate negatives and positives
    negatives = [-x for x in arr if x < 0]
    positives = [x for x in arr if x >= 0]
    
    # Sort positives
    if positives:
        positives = _radix_sort_positive(positives, base)
    
    # Sort negatives (by absolute value, then reverse)
    if negatives:
        negatives = _radix_sort_positive(negatives, base)
        negatives = [-x for x in reversed(negatives)]
    
    return negatives + positives


def _radix_sort_positive(arr: List[int], base: int) -> List[int]:
    """
    Helper for positive integers only.
    """
    if not arr:
        return []
    
    # Find maximum to determine number of digits
    max_val = max(arr)
    
    # Process each digit
    exp = 1  # 10^0, 10^1, 10^2, ...
    while max_val // exp > 0:
        # Sort by digit at position exp using counting sort
        arr = _counting_sort_by_digit(arr, exp, base)
        exp *= base
    
    return arr


def _counting_sort_by_digit(arr: List[int], 
                            exp: int, 
                            base: int) -> List[int]:
    """
    Stable counting sort for specific digit.
    
    exp: 10^i where i is current digit position (0-indexed from right)
    """
    n = len(arr)
    output = [0] * n
    count = [0] * base  # Digits 0 to base-1
    
    # Count occurrences of each digit at position exp
    for num in arr:
        digit = (num // exp) % base
        count[digit] += 1
    
    # Compute prefix sums (cumulative counts)
    for i in range(1, base):
        count[i] += count[i - 1]
    
    # Build output array (stable - go right to left)
    for i in range(n - 1, -1, -1):
        digit = (arr[i] // exp) % base
        output[count[digit] - 1] = arr[i]
        count[digit] -= 1
    
    return output


def radix_sort_strings(arr: List[str], 
                       max_length: int = None) -> List[str]:
    """
    Radix sort for strings (LSD).
    
    Pads shorter strings or processes only up to max_length.
    """
    if not arr:
        return []
    
    if max_length is None:
        max_length = max(len(s) for s in arr)
    
    # Work with list of chars for each position
    result = arr.copy()
    
    # Sort from rightmost character to leftmost
    for pos in range(max_length - 1, -1, -1):
        # Use counting sort with ASCII (256 possible chars)
        # Handle strings shorter than pos by treating as char 0
        result = _counting_sort_by_char(result, pos)
    
    return result


def _counting_sort_by_char(arr: List[str], pos: int) -> List[str]:
    """
    Counting sort strings by character at position pos.
    """
    n = len(arr)
    output = [''] * n
    count = [0] * 257  # 256 ASCII + 1 for "no char" (shorter strings)
    
    # Count occurrences
    for s in arr:
        if pos < len(s):
            char_code = ord(s[pos]) + 1  # +1 to avoid 0 (used for padding)
        else:
            char_code = 0  # Shorter strings come first
        count[char_code] += 1
    
    # Prefix sums
    for i in range(1, 257):
        count[i] += count[i - 1]
    
    # Build output (stable)
    for i in range(n - 1, -1, -1):
        s = arr[i]
        if pos < len(s):
            char_code = ord(s[pos]) + 1
        else:
            char_code = 0
        
        output[count[char_code] - 1] = s
        count[char_code] -= 1
    
    return output


def demonstrate_radix_sort():
    """
    Demonstrate radix sort.
    """
    print("\nRadix Sort (LSD) Demonstration")
    print("=" * 70)
    
    # Test case 1: Positive integers
    arr1 = [170, 45, 75, 90, 802, 24, 2, 66]
    print(f"Input:  {arr1}")
    sorted1 = radix_sort_lsd(arr1)
    print(f"Sorted: {sorted1}")
    
    # Test case 2: With negatives
    arr2 = [-170, 45, -75, 90, -2, 24, 2, 66]
    print(f"\nWith negatives: {arr2}")
    sorted2 = radix_sort_lsd(arr2)
    print(f"Sorted:        {sorted2}")
    
    # Test case 3: Strings
    words = ["dog", "cat", "bat", "apple", "ant", "zebra"]
    print(f"\nStrings: {words}")
    sorted_words = radix_sort_strings(words)
    print(f"Sorted:  {sorted_words}")
    
    print("""
    
    LSD Radix Sort Characteristics:
    ─────────────────────────────────────────────────────────────────────
    
    Advantages:
      • O(n) time for fixed-length keys (d is constant)
      • Stable sort
      • No comparisons between keys (only digit extraction)
      • Cache friendly (sequential access)
    
    Disadvantages:
      • Requires extra space O(n)
      • Not in-place
      • Slower than QuickSort for small n (constant factors)
      • Requires uniform key length (or handling variable length)
    
    Use Cases:
      • Sorting integers (especially 32-bit or 64-bit)
      • Sorting fixed-length strings (e.g., phone numbers, SSNs)
      • Sorting dates (YYYYMMDD format)
      • Parallel sorting (each digit pass is independent)
    
    Why LSD over MSD?
      • LSD is simpler (no recursion)
      • LSD guaranteed O(n) for fixed length
      • MSD better for variable length (can stop early)
    """)


demonstrate_radix_sort()
```

---

### **9.3.2 MSD (Most Significant Digit) Radix Sort**

```python
def radix_sort_msd(arr: List[str], 
                   alphabet_size: int = 256) -> List[str]:
    """
    MSD (Most Significant Digit) Radix Sort.
    
    Recursive approach similar to QuickSort but by characters.
    
    Algorithm:
      1. Partition array by first character (using counting sort)
      2. Recursively sort each partition by next character
      3. Stop when partition size is small (use insertion sort)
    
    Time: O(n × L) where L is average string length
    Space: O(n + L) recursion stack
    
    Better than LSD for:
      • Variable length strings (can skip empty partitions)
      • When strings differ early (prune recursion)
    """
    if not arr:
        return []
    
    # Convert to list for in-place modifications
    result = arr.copy()
    _msd_sort_recursive(result, 0, len(result) - 1, 0, alphabet_size)
    return result


def _msd_sort_recursive(arr: List[str], 
                        low: int, 
                        high: int, 
                        char_pos: int,
                        alphabet_size: int) -> None:
    """
    Recursively sort subarray [low..high] by character at position char_pos.
    """
    if low >= high:
        return
    
    # Use insertion sort for small subarrays (optimization)
    if high - low <= 10:
        _insertion_sort_by_char(arr, low, high, char_pos)
        return
    
    # Counting sort by current character
    # We need to partition in-place or use auxiliary arrays
    
    # Create buckets for each character + 1 for "end of string"
    # char 0: end of string (shortest strings)
    # char 1-256: actual characters
    
    # Count frequencies
    count = [0] * (alphabet_size + 2)  # +2 for end-of-string and indexing offset
    
    for i in range(low, high + 1):
        if char_pos < len(arr[i]):
            char = ord(arr[i][char_pos]) + 1  # +1 to reserve 0 for end-of-string
        else:
            char = 0  # End of string marker
        count[char + 1] += 1  # +1 for prefix sum calculation
    
    # Convert to prefix sums (indices)
    for i in range(1, len(count)):
        count[i] += count[i - 1]
    
    # Create auxiliary array for stable partitioning
    aux = [''] * (high - low + 1)
    
    for i in range(low, high + 1):
        if char_pos < len(arr[i]):
            char = ord(arr[i][char_pos]) + 1
        else:
            char = 0
        aux[count[char]] = arr[i]
        count[char] += 1
    
    # Copy back
    for i in range(low, high + 1):
        arr[i] = aux[i - low]
    
    # Recursively sort each non-empty bucket
    # Skip bucket 0 (end-of-string strings are done)
    start = low + count[0] - low  # First index after empty strings
    
    for i in range(1, len(count) - 1):
        end = low + count[i] - low - 1
        if start <= end:
            _msd_sort_recursive(arr, start, end, char_pos + 1, alphabet_size)
        start = end + 1


def _insertion_sort_by_char(arr: List[str], 
                            low: int, 
                            high: int, 
                            char_pos: int) -> None:
    """
    Insertion sort for small subarrays in MSD sort.
    """
    for i in range(low + 1, high + 1):
        key = arr[i]
        j = i - 1
        
        while j >= low and _compare_strings(arr[j], key, char_pos) > 0:
            arr[j + 1] = arr[j]
            j -= 1
        
        arr[j + 1] = key


def _compare_strings(s1: str, s2: str, char_pos: int) -> int:
    """
    Compare strings starting from char_pos.
    Returns: negative if s1 < s2, 0 if equal, positive if s1 > s2
    """
    # Skip to char_pos
    if char_pos >= len(s1):
        return -1 if char_pos < len(s2) else 0
    if char_pos >= len(s2):
        return 1
    
    # Compare from char_pos onwards
    return (s1[char_pos:] > s2[char_pos:]) - (s1[char_pos:] < s2[char_pos:])


def demonstrate_msd_sort():
    """
    Demonstrate MSD radix sort.
    """
    print("\nMSD Radix Sort Demonstration")
    print("=" * 70)
    
    words = ["she", "sells", "seashells", "by", "the", "seashore",
             "the", "shells", "she", "sells", "are", "surely", "seashells"]
    
    print(f"Words: {words}")
    sorted_words = radix_sort_msd(words)
    print(f"Sorted: {sorted_words}")
    
    print("""
    
    LSD vs MSD Radix Sort:
    ─────────────────────────────────────────────────────────────────────
    
    LSD (Least Significant Digit):
      • Process digits right to left
      • Requires d passes over entire data
      • Better for fixed-length keys
      • Simpler implementation (iterative)
      • Guaranteed O(n × d)
    
    MSD (Most Significant Digit):
      • Process digits left to right
      • Recursive, like QuickSort by characters
      • Better for variable-length strings
      • Can terminate early (if strings differ at start)
      • Cache performance may suffer (random access in recursion)
    
    Hybrid Approach (Tries):
      • Use trie data structure (Chapter 25)
      • Sort by traversing trie in order
      • O(n × L) time, O(n × L) space
      • Natural fit for MSD philosophy
    """)


demonstrate_msd_sort()
```

---

## **9.4 Bucket Sort**

Bucket Sort assumes input is uniformly distributed over a range. It distributes elements into buckets, sorts each bucket (usually with Insertion Sort), then concatenates.

```python
def bucket_sort(arr: List[float], 
                num_buckets: int = None,
                bucket_algorithm: str = 'insertion') -> List[float]:
    """
    Bucket Sort for uniformly distributed floating point numbers.
    
    Algorithm:
      1. Create n empty buckets (lists)
      2. Scatter: Insert each element into appropriate bucket
         - Bucket index = floor(n × element) for [0, 1) range
      3. Sort each bucket individually
      4. Gather: Concatenate all buckets
    
    Time Complexity:
      Best/Average: O(n) when data uniformly distributed
      Worst: O(n²) when all elements in one bucket
             (mitigated by using good hash function or more buckets)
    
    Space: O(n) for buckets
    
    Stability: Yes (if bucket sort is stable)
    
    Args:
        arr: List of floats in range [0, 1) (will normalize if outside)
        num_buckets: Number of buckets (default n)
        bucket_algorithm: 'insertion' or 'quicksort' for individual buckets
    
    Returns:
        Sorted list
    """
    n = len(arr)
    if n <= 1:
        return arr.copy() if arr else []
    
    # Normalize if necessary (find min/max)
    min_val = min(arr)
    max_val = max(arr)
    range_size = max_val - min_val
    
    if range_size == 0:
        return arr.copy()  # All elements equal
    
    # Normalize to [0, 1)
    normalized = [(x - min_val) / range_size for x in arr]
    
    # Determine number of buckets
    if num_buckets is None:
        num_buckets = n
    
    # Create buckets
    buckets = [[] for _ in range(num_buckets)]
    
    # Scatter elements into buckets
    for num in normalized:
        # Calculate bucket index
        idx = int(num * num_buckets)
        # Handle edge case where num == 1.0
        if idx >= num_buckets:
            idx = num_buckets - 1
        buckets[idx].append(num)
    
    # Sort individual buckets
    for i in range(num_buckets):
        if bucket_algorithm == 'insertion':
            _insertion_sort(buckets[i])
        else:
            buckets[i].sort()  # Python's Timsort
    
    # Gather results and denormalize
    result = []
    for bucket in buckets:
        for num in bucket:
            # Denormalize back to original range
            original = num * range_size + min_val
            result.append(original)
    
    return result


def _insertion_sort(arr: List[float]) -> None:
    """
    Standard insertion sort for individual buckets.
    """
    for i in range(1, len(arr)):
        key = arr[i]
        j = i - 1
        while j >= 0 and arr[j] > key:
            arr[j + 1] = arr[j]
            j -= 1
        arr[j + 1] = key


def bucket_sort_integers(arr: List[int], 
                         num_digits: int = 2) -> List[int]:
    """
    Bucket sort variant for integers using most significant digits.
    
    Creates buckets based on leading digits, then sorts within.
    """
    if not arr:
        return []
    
    # Find range
    max_val = max(abs(x) for x in arr)
    min_val = min(arr)
    
    # Determine bucket size based on number of digits
    bucket_range = 10 ** num_digits
    
    # Create buckets (handle negative and positive separately)
    neg_buckets = {}
    pos_buckets = {}
    
    for num in arr:
        if num < 0:
            # For negatives, bucket by absolute value
            bucket_idx = abs(num) // bucket_range
            if bucket_idx not in neg_buckets:
                neg_buckets[bucket_idx] = []
            neg_buckets[bucket_idx].append(num)
        else:
            bucket_idx = num // bucket_range
            if bucket_idx not in pos_buckets:
                pos_buckets[bucket_idx] = []
            pos_buckets[bucket_idx].append(num)
    
    # Sort each bucket and concatenate
    result = []
    
    # Add negatives in reverse order (most negative first)
    for idx in sorted(neg_buckets.keys(), reverse=True):
        bucket = neg_buckets[idx]
        bucket.sort()
        result.extend(bucket)
    
    # Add positives in order
    for idx in sorted(pos_buckets.keys()):
        bucket = pos_buckets[idx]
        bucket.sort()
        result.extend(bucket)
    
    return result


def demonstrate_bucket_sort():
    """
    Demonstrate bucket sort.
    """
    print("\nBucket Sort Demonstration")
    print("=" * 70)
    
    # Test case 1: Uniform floats in [0, 1)
    import random
    random.seed(42)
    arr1 = [random.random() for _ in range(20)]
    print(f"Uniform floats (first 5): {arr1[:5]}")
    sorted1 = bucket_sort(arr1)
    print(f"Sorted (first 5):         {sorted1[:5]}")
    print(f"Is sorted: {all(sorted1[i] <= sorted1[i+1] for i in range(len(sorted1)-1))}")
    
    # Test case 2: Non-uniform (worst case for bucket sort)
    arr2 = [0.1] * 20  # All same bucket
    print(f"\nWorst case (all same): {arr2}")
    sorted2 = bucket_sort(arr2)
    print(f"Sorted: {sorted2}")
    
    # Test case 3: Integers
    arr3 = [random.randint(0, 999) for _ in range(20)]
    print(f"\nIntegers: {arr3}")
    sorted3 = bucket_sort_integers(arr3, num_digits=1)
    print(f"Sorted:   {sorted3}")
    
    print("""
    
    Bucket Sort Analysis:
    ─────────────────────────────────────────────────────────────────────
    
    Assumption: Uniform distribution over [0, 1)
    
    Mathematical Analysis:
      • n elements into n buckets
      • Expected elements per bucket: 1
      • Sorting each bucket: O(1) expected
      • Total: O(n) expected time
    
    Worst Case: O(n²)
      • When all elements fall into one bucket
      • Mitigation: Use more buckets (n log n buckets → O(n log n) worst)
      • Or use QuickSort for individual buckets
    
    Use Cases:
      • Floating point numbers in known range
      • Data known to be uniformly distributed
      • Histogram-based sorting
      • External sorting (buckets fit in memory individually)
    
    Variations:
      • Histogram Sort: Count frequencies, then output (no individual sorting)
      • Proxmap Sort: Maps keys to buckets using interpolation
    """)


demonstrate_bucket_sort()
```

---

## **9.5 External Sorting**

When data is too large to fit in memory, we use **External Sorting**. The most common approach is **External Merge Sort**.

### **9.5.1 K-Way Merge External Sort**

```python
import heapq
import os
from typing import List, Iterator, TextIO
import tempfile

class ExternalSorter:
    """
    External Merge Sort for data that doesn't fit in memory.
    
    Algorithm:
      1. Phase 1 - Sort Chunks:
         • Read M lines at a time (M = memory capacity)
         • Sort in memory using internal sort
         • Write to temporary file (run)
      
      2. Phase 2 - K-Way Merge:
         • Open all temporary files
         • Use min-heap to find smallest among K files
         • Write to output, replenish from corresponding file
         • Repeat until all files exhausted
    
    Time Complexity:
      • External sorting: O(n log n) comparisons
      • But dominated by I/O cost
      • Number of passes: O(log_{M/B} (N/B)) where:
        - N = total records, M = memory size, B = block size
    
    Space: O(M) in memory (plus O(N/M) temporary files)
    """
    
    def __init__(self, memory_limit: int = 1000):
        """
        Args:
            memory_limit: Maximum number of records to hold in memory
        """
        self.memory_limit = memory_limit
        self.temp_files = []
    
    def sort_file(self, input_file: str, output_file: str) -> None:
        """
        Sort input file and write to output file.
        
        Assumes file contains one record per line.
        """
        # Phase 1: Create sorted runs
        self._create_runs(input_file)
        
        # Phase 2: K-way merge
        self._k_way_merge(output_file)
        
        # Cleanup
        self._cleanup()
    
    def _create_runs(self, input_file: str) -> None:
        """
        Phase 1: Split input into sorted chunks (runs).
        """
        with open(input_file, 'r') as f:
            chunk = []
            for line in f:
                chunk.append(line.strip())
                
                if len(chunk) >= self.memory_limit:
                    self._write_run(chunk)
                    chunk = []
            
            # Last chunk
            if chunk:
                self._write_run(chunk)
    
    def _write_run(self, chunk: List[str]) -> None:
        """
        Sort chunk and write to temporary file.
        """
        chunk.sort()  # Internal sort
        
        # Create temp file
        temp = tempfile.NamedTemporaryFile(mode='w', 
                                           delete=False, 
                                           suffix='.run')
        for item in chunk:
            temp.write(item + '\n')
        temp.close()
        
        self.temp_files.append(temp.name)
    
    def _k_way_merge(self, output_file: str) -> None:
        """
        Phase 2: Merge K sorted files using min-heap.
        
        Uses heapq.merge for efficient K-way merge.
        """
        # Open all temp files
        files = [open(f, 'r') for f in self.temp_files]
        
        # Create iterators
        iterators = [self._file_iterator(f) for f in files]
        
        with open(output_file, 'w') as out:
            # K-way merge using heapq.merge (generates sorted iterator)
            for item in heapq.merge(*iterators):
                out.write(item + '\n')
        
        # Close files
        for f in files:
            f.close()
    
    def _file_iterator(self, file_obj: TextIO) -> Iterator[str]:
        """
        Generator to yield lines from file one at a time.
        """
        for line in file_obj:
            yield line.strip()
    
    def _cleanup(self) -> None:
        """
        Remove temporary files.
        """
        for temp_file in self.temp_files:
            try:
                os.remove(temp_file)
            except OSError:
                pass
        self.temp_files = []


def external_sort_simulation():
    """
    Simulate external sorting with in-memory example.
    """
    print("\nExternal Sorting (K-Way Merge) Demonstration")
    print("=" * 70)
    
    # Simulate large dataset
    import random
    random.seed(42)
    
    # Generate "large" dataset (would be on disk in reality)
    data = [random.randint(1, 10000) for _ in range(50)]
    memory_limit = 10  # Can only hold 10 items at a time
    
    print(f"Total records: {len(data)}")
    print(f"Memory limit: {memory_limit} records")
    print(f"Requires: {len(data) // memory_limit} runs + merge")
    
    # Phase 1: Create sorted runs
    runs = []
    for i in range(0, len(data), memory_limit):
        chunk = data[i:i+memory_limit]
        chunk.sort()
        runs.append(chunk)
        print(f"Run {i//memory_limit + 1}: {chunk}")
    
    # Phase 2: K-way merge using heap
    print(f"\nMerging {len(runs)} runs using min-heap...")
    
    # Use heapq.merge for K-way merge
    merged = list(heapq.merge(*runs))
    print(f"Result: {merged[:20]}... (first 20)")
    print(f"Is sorted: {all(merged[i] <= merged[i+1] for i in range(len(merged)-1))}")
    
    print("""
    
    External Sorting Analysis:
    ─────────────────────────────────────────────────────────────────────
    
    Why External Sorting?
      • Data size (N) >> Memory size (M)
      • Minimize disk I/O (slowest operation)
      • Sequential access preferred over random access
    
    K-Way Merge Optimization:
      • Standard merge sort: 2-way (merge 2 files at a time)
      • K-way: Merge K files simultaneously
      • Reduces number of passes from log2(N/M) to logK(N/M)
      • Use min-heap for O(log K) per element (vs O(K) for linear scan)
    
    Replacement Selection:
      • Advanced technique to create longer initial runs
      • Uses heap to produce runs of size ~2M on average
      • Reduces number of passes needed
    
    Polyphase Merge:
      • Optimal merge pattern when number of runs != power of K
      • Minimizes dummy runs (empty runs)
      • Used in tape sorting (historical) and some database systems
    
    Buffer Management:
      • Read blocks of size B (block size) to minimize I/O
      • Double buffering: read next block while processing current
    
    Real-world Applications:
      • Database systems (ORDER BY on large tables)
      • Log processing (sorting terabyte-scale logs)
      • MapReduce (external sort between map and reduce phases)
      • Search engines (inverted index construction)
    """)


external_sort_sort_simulation = external_sort_simulation  # Fix typo
external_sort_sort_simulation()
```

**Note:** The function was misspelled in the call, fixing to `external_sort_simulation`.

---

## **9.6 Summary and Key Takeaways**

```python
def non_comparison_summary():
    """
    Summary of non-comparison sorting algorithms.
    """
    
    print("Non-Comparison Sorting Summary")
    print("=" * 70)
    
    print("""
    Algorithm       │ Time        │ Space    │ Stable │ Constraints
    ────────────────┼─────────────┼──────────┼────────┼─────────────────────
    Counting Sort   │ O(n + k)    │ O(n + k) │ Yes    │ Integers in [0,k]
    Radix Sort (LSD)│ O(d(n + k)) │ O(n + k) │ Yes    │ Fixed-length keys
    Radix Sort (MSD)│ O(d(n + k)) │ O(n + dk)│ Yes    │ Variable strings
    Bucket Sort     │ O(n) avg    │ O(n)     │ Yes    │ Uniform distribution
    External Sort   │ O(n log n)  │ O(M)     │ Yes    │ N >> M (memory)
    
    Where:
      • n = number of elements
      • k = range of values (for counting sort)
      • d = number of digits/maximum length
      • M = memory size
    
    ─────────────────────────────────────────────────────────────────────
    
    Decision Tree:
    
    Are keys integers in small range?
      Yes → Counting Sort (O(n + k))
      No  → Are they fixed-length strings/numbers?
              Yes → Radix Sort (O(n × d))
              No  → Are they uniformly distributed floats?
                      Yes → Bucket Sort (O(n) expected)
                      No  → Is data too large for memory?
                              Yes → External Merge Sort
                              No  → Comparison Sort (QuickSort/TimSort)
    
    ─────────────────────────────────────────────────────────────────────
    
    Hybrid Strategies (Industry Standard):
    
    Python's sorted() (Timsort):
      • Hybrid of Merge Sort + Insertion Sort
      • O(n log n) worst case, O(n) best case
      • Stable, adaptive
    
    C++ std::sort (IntroSort):
      • Hybrid of QuickSort + HeapSort + Insertion Sort
      • O(n log n) worst case
      • Not stable
    
    Java Arrays.sort():
      • Primitives: Dual-Pivot QuickSort
      • Objects: Timsort (stable)
    
    Database Sorting:
      • In-memory: QuickSort or Timsort
      • External: K-way Merge Sort with replacement selection
    """)

non_comparison_summary()
```

---

## **9.7 Practice Problems**

### **Problem 1: Sort Colors (Dutch National Flag)**
Given an array with n objects colored red, white, or blue (represented as 0, 1, 2), sort them in-place so that objects of the same color are adjacent. Use the **Dutch National Flag algorithm** (three-way partitioning) with O(n) time and O(1) space.

**Hint**: Maintain three pointers: `low` (boundary of 0s), `mid` (current), `high` (boundary of 2s).

### **Problem 2: Maximum Gap**
Given an unsorted array, find the maximum difference between successive elements in the sorted form. Solve in O(n) time using **Bucket Sort** concept (pigeonhole principle).

**Hint**: If you have n numbers, the max gap is at least `(max - min) / (n - 1)`. Use buckets of this size.

### **Problem 3: Sort Array by Parity**
Separate even and odd numbers using a stable partition (similar to counting sort philosophy but with two buckets).

### **Problem 4: Intersection of Two Arrays**
Given two arrays, compute their intersection. Each element in the result must be unique. Use **Counting Sort** concept to achieve O(n + m) time.

### **Problem 5: External Sort with Duplicates**
Modify the external sort implementation to handle duplicate keys efficiently (remove duplicates during merge using the heap property).

---

## **9.8 Further Reading**

1. **"Introduction to Algorithms" (CLRS)** - Chapter 8: Sorting in Linear Time (Counting, Radix, Bucket Sort)
2. **"Algorithm Design"** by Kleinberg & Tardos - Chapter on Linear Time Sorting
3. **"The Art of Computer Programming, Vol 3"** by Knuth - Section on Radix Sorting
4. **"External Sorting"** research papers on Polyphase Merging and Replacement Selection
5. **"Engineering Radix Sort"** by McIlroy et al. - Optimized implementations for real-world data

---

> **Coming in Chapter 10**: **Searching Algorithms** - We will explore efficient searching techniques including Linear Search, Binary Search (and its variants), Interpolation Search, and applications in rotated arrays. You'll master the art of finding elements in O(log n) time and understand when to use each search strategy.

---

**End of Chapter 9**

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='8. comparison_based_sorting.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='10. searching_algorithms.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
